In [ ]:
from python_speech_features import mfcc #Mel Frequency Copstral Coefficient
import scipy.io.wavfile as wav
import numpy as np

from tempfile import TemporaryFile

import os
import pickle
import random
import operator
import math

In [1]:
#function to perform actual distance calculations between features
def distance(instance1, instance2, k):
    distance = 0
    mm1 = instance1[0]
    cm1 = instance1[1]
    mm2 = instance2[0]
    cm2 = instance2[1]
    distance = np.trace(np.dot(np.linalg.inv(cm2), cm1))
    distance += (np.dot(np.dot((mm2-mm1).transpose(), np.linalg.inv(cm2)), mm2-mm1))
    distance += np.log(np.linalg.det(cm2)) - np.log(np.linalg.det(cm1))
    distance -= k
    return distance

In [2]:
# function to get the distance between feature vectors and find neighbors
def getNeighbors(trainingSet, instance, k):
    distance = [] #distance list
    #calculates all possible distance among items or instance
    for x in range(len(trainingSet)):
        dist = distance(trainingSet[x], instance, k) + distance(instance, trainingSet[x], k)
        distance.append((trainingSet[x][2], dist))

    #sorting the data   
    distance.sort(key=operator.itemgetter(1))
    #for k nearest neighbors
    neighbors = [] #new list
    for x in range(k):
        neighbors.append(distance[x][0])
    return neighbors


In [3]:
# identify the class of the neighbors
def nearestClass(neighbors):
    classVote = {} #set or dictionary

    for x in range(len(neighbors)):
        response = neighbors[x]
        if response in classVote:
            classVote[response] += 1
        else:
            classVote[response] = 1
    sorter = sorted(classVote.items(), key= operator.itemgetter(1), reverse=True)

    return sorter[0][0] #sorted array

In [4]:
# function to evaluate the model
def getAccuracy(testSet, prediction):
    correct = 0
    for x in range(len(testSet)):
        if testSet[x][-1] == prediction[x]:
            correct += 1
    return (1.0 * correct) / len(testSet) 

In [6]:
# directory that hold the dataset
#directory = "./myproject/Data/genres"
directory = "C:/Users/Asus/myproject/Data/genres/"

In [1]:
#binary file where we will collect all the features extractred
#using mfcc
f = open("my.dat", 'wb')
i = 0
# considering 10 neighbors
for folder in os.listdir(directory):
    i += 1
    if i == 11:
        break
    for file in os.listdir(directory+folder):
        try:
            (rate, sig) = wav.read(directory+folder+"/"+file)
            mfcc_feat = mfcc(sig, rate, winlen=0.020, appendEnergy=False)
            covariance = np.cov(np.matrix.transpose(mfcc_feat))
            mean_matrix = mfcc_feat.mean(0)
            feature = (mean_matrix, covariance, i)
            pickle.dump(feature, f)
        except Exception as e:
            print('Got an exception: ', e, 'in folder: ', folder, 'filename: ', file)
            
f.close()

NameError: name 'directory' is not defined

In [18]:
# split dataset into training and testing sets respectively
dataset = []
def loadDataset(filenam, split, trSet, teSet):
    with open(filename, 'rb') as f:
        while True:
           try:
               dataset.append(pickle.load(f))
            except EOFError:
                f.close()
                break
    for x in range(len(dataset)):
        if random.random() < split:
            trSet.append(dataset[x])
        else:
            teSet.append(dataset[x])

trainingSet = []
testSet = []
loadDataset("my.dat", 0.66, trainingSet, testSet)

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 8)

In [10]:
# making prediction using knn
leng = len(testSet)
predictions = []
for x in range(leng):
    predictions.append(nearestClass(getNeighbors(trainingSet, testSet[x], 5)))

accuracy1 = getAccuracy(testSet, predictions)
print(accuracy1)

NameError: name 'testSet' is not defined